In [30]:
using Pkg
Pkg.activate("/home/mathow/Documents/EnKF/")

"/home/mathow/Documents/EnKF/Project.toml"

In [31]:
using Revise
using EnKF
using Distributions
using LinearAlgebra
using Lazy

In [32]:
f = PropagationFunction()
m = MeasurementFunction()
z = RealMeasurementFunction()

RealMeasurementFunction()

In [33]:
function (::PropagationFunction)(t::Float64, ENS::EnsembleState{N, TS}) where {N, TS}
    return ENS  
end

In [34]:
function (::MeasurementFunction)(t::Float64, mENS::EnsembleState{N, NZ, TZ},  ENS::EnsembleState{N, TS}) where {N, TS, NZ, TZ}
    fill!(mENS, 1.01*ones(NZ))
    return mENS
end

ErrorException: too many parameters for type

In [35]:
function (::MeasurementFunction)(t::Float64, S::Array{TS,1}) where {TS}
    return 1.05*ones(NZ)
end

In [36]:
function (::RealMeasurementFunction)(t::Float64, zENS::EnsembleState{N, TZ}) where {N, TZ} 
    fill!(zENS, 1.01*ones(NZ))
    return zENS
end

In [37]:
# constantly(1)

#### Ensemble Kalman filter

In [41]:
N = 10
NS = 3
TS = Float64
NZ = 5
TZ = Float64
β = MultiplicativeInflation(NS, 1.01)
ϵ = AdditiveInflation(NZ)
isinflated = true
isaugmented = false

false

In [42]:
enkf = ENKF{N, TS, NZ, TZ}(f, β, m, z, ϵ, isinflated, isaugmented)

ENKF{10,Float64,5,Float64}(PropagationFunction(), MultiplicativeInflation{3}(1.01), MeasurementFunction(), RealMeasurementFunction(), AdditiveInflation{5}(IsoNormal(
dim: 5
μ: [0.0, 0.0, 0.0, 0.0, 0.0]
Σ: [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0]
)
), true, false)

In [43]:
ens = EnsembleState(N, zeros(NS))

EnsembleState{10,Array{Float64,1}}(Array{Float64,1}[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])

#### Initialize each ensemble member

In [44]:
initialize!(ens,zeros(NS))

EnsembleState{10,Array{Float64,1}}(Array{Float64,1}[[-1.14367, -0.0277021, 0.0853983], [0.884677, -0.879113, 1.42037], [-1.51733, -0.844301, -1.41058], [-2.79547, -1.11874, -0.256039], [-0.966595, -0.823062, -0.690766], [-0.598018, -0.323214, -0.346194], [-0.277284, -0.416709, 0.693069], [-1.21544, 0.782157, -1.94301], [3.66612, 0.0805916, -1.01547], [0.403568, 0.982476, -0.39372]])

#### Propagate each ensemble member

In [45]:
enkf.f(0.0, ens)

EnsembleState{10,Array{Float64,1}}(Array{Float64,1}[[-1.14367, -0.0277021, 0.0853983], [0.884677, -0.879113, 1.42037], [-1.51733, -0.844301, -1.41058], [-2.79547, -1.11874, -0.256039], [-0.966595, -0.823062, -0.690766], [-0.598018, -0.323214, -0.346194], [-0.277284, -0.416709, 0.693069], [-1.21544, 0.782157, -1.94301], [3.66612, 0.0805916, -1.01547], [0.403568, 0.982476, -0.39372]])

#### Apply convariance inflation

In [46]:
if isinflated ==true
        enkf.A(ens)
end

EnsembleState{10,Array{Float64,1}}(Array{Float64,1}[[-1.15155, -0.0253915, 0.0901092], [0.897083, -0.885317, 1.43843], [-1.52894, -0.850156, -1.42083], [-2.81987, -1.12734, -0.254743], [-0.972701, -0.828706, -0.693817], [-0.600439, -0.323858, -0.345799], [-0.276498, -0.418288, 0.703856], [-1.22403, 0.792566, -1.95858], [3.70634, 0.0839851, -1.02177], [0.411163, 0.994888, -0.3938]])

In [47]:
ens

EnsembleState{10,Array{Float64,1}}(Array{Float64,1}[[-1.15155, -0.0253915, 0.0901092], [0.897083, -0.885317, 1.43843], [-1.52894, -0.850156, -1.42083], [-2.81987, -1.12734, -0.254743], [-0.972701, -0.828706, -0.693817], [-0.600439, -0.323858, -0.345799], [-0.276498, -0.418288, 0.703856], [-1.22403, 0.792566, -1.95858], [3.70634, 0.0839851, -1.02177], [0.411163, 0.994888, -0.3938]])

#### Compute mean and deviation

In [48]:
Ŝ = deepcopy(mean(ens))

3-element Array{Float64,1}:
 -0.35594398274321193
 -0.2587614500210512 
 -0.3856941530052703 

In [49]:
size(ens)

(10, (3,))

In [51]:
length(ens)

10

In [54]:
ensfluc = EnsembleState(ens.S)

deviation(ensfluc, ens)
A′ = hcat(ensfluc)

3×10 Array{Float64,2}:
 -0.795605   1.25303   -1.173     …  -0.868089   4.06229    0.767107 
  0.23337   -0.626555  -0.591394      1.05133    0.342747   1.25365  
  0.475803   1.82412   -1.03513      -1.57289   -0.636076  -0.0081062

In [60]:
mean(A′[3,:])

-5.880712583561376e-17

ArgumentError: Package Tokenize not found in current path:
- Run `import Pkg; Pkg.add("Tokenize")` to install the Tokenize package.

Stacktrace:
 [1] require(::Module, ::Symbol) at ./loading.jl:823
 [2] eval at ./boot.jl:328 [inlined]
 [3] prepare_thunk(::Module, ::Expr, ::Bool) at /home/mathow/.julia/packages/JuliaInterpreter/Or807/src/construct.jl:355
 [4] prepare_thunk(::Module, ::Expr, ::Bool) at /home/mathow/.julia/packages/JuliaInterpreter/Or807/src/construct.jl:361
 [5] prepare_thunk at /home/mathow/.julia/packages/JuliaInterpreter/Or807/src/construct.jl:347 [inlined]
 [6] #methods_by_execution!#9(::Base.Iterators.Pairs{Symbol,Bool,Tuple{Symbol},NamedTuple{(:define,),Tuple{Bool}}}, ::Function, ::Any, ::Revise.CodeTrackingMethodInfo, ::Dict{Module,Array{Expr,1}}, ::Module, ::Expr) at /home/mathow/.julia/packages/Revise/agmgx/src/lowered.jl:47
 [7] #methods_by_execution! at ./none:0 [inlined]
 [8] #eval_with_signatures#59(::Bool, ::Base.Iterators.Pairs{Union{},Union{},Tuple{},Name

LoadError: UndefVarError: @default not defined
Stacktrace:
 [1] top-level scope
 [2] lower at ./meta.jl:68 [inlined]
 [3] prepare_thunk(::Module, ::Expr, ::Bool) at /home/mathow/.julia/packages/JuliaInterpreter/Or807/src/construct.jl:360
 [4] prepare_thunk at /home/mathow/.julia/packages/JuliaInterpreter/Or807/src/construct.jl:347 [inlined]
 [5] #methods_by_execution!#9(::Base.Iterators.Pairs{Symbol,Bool,Tuple{Symbol},NamedTuple{(:define,),Tuple{Bool}}}, ::Function, ::Any, ::Revise.CodeTrackingMethodInfo, ::Dict{Module,Array{Expr,1}}, ::Module, ::Expr) at /home/mathow/.julia/packages/Revise/agmgx/src/lowered.jl:47
 [6] #methods_by_execution! at ./none:0 [inlined]
 [7] #eval_with_signatures#59(::Bool, ::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}, ::Function, ::Module, ::Expr) at /home/mathow/.julia/packages/Revise/agmgx/src/Revise.jl:344
 [8] #eval_with_signatures at /home/mathow/.julia/packages/Revise/agmgx/src/Revise.jl:0 [inlined]
 [9] #instantiate_sigs!#60(

        if is_number(ret) && last(ret.val) == '.'
            #= /home/mathow/.julia/packages/CSTParser/i8cPl/src/CSTParser.jl:93 =#
            ps.errored = true
            #= /home/mathow/.julia/packages/CSTParser/i8cPl/src/CSTParser.jl:94 =#
            ret = mErrorToken(ret, CannotJuxtapose)
        end
        #= /home/mathow/.julia/packages/CSTParser/i8cPl/src/CSTParser.jl:96 =#
        op = mOPERATOR(0, 0, Tokens.STAR, false)
        #= /home/mathow/.julia/packages/CSTParser/i8cPl/src/CSTParser.jl:97 =#
        ret = parse_operator(ps, ret, op)
    elseif #= /home/mathow/.julia/packages/CSTParser/i8cPl/src/CSTParser.jl:98 =# (ret.typ === x_Str || ret.typ === x_Cmd) && (ps.nt).kind == Tokens.IDENTIFIER
        #= /home/mathow/.julia/packages/CSTParser/i8cPl/src/CSTParser.jl:99 =#
        arg = mIDENTIFIER(next(ps))
        #= /home/mathow/.julia/packages/CSTParser/i8cPl/src/CSTParser.jl:100 =#
        push!(ret, mLITERAL(arg.fullspan, arg.span, val(ps.t, ps), Tokens.STRING))
   

#= /home/mathow/.julia/packages/CSTParser/i8cPl/src/CSTParser.jl:154 =# @addctx :paren function parse_paren(ps::ParseState)
        #= /home/mathow/.julia/packages/CSTParser/i8cPl/src/CSTParser.jl:155 =#
        args = EXPR[mPUNCTUATION(ps)]
        #= /home/mathow/.julia/packages/CSTParser/i8cPl/src/CSTParser.jl:156 =#
        #= /home/mathow/.julia/packages/CSTParser/i8cPl/src/CSTParser.jl:156 =# @closeparen ps #= /home/mathow/.julia/packages/CSTParser/i8cPl/src/CSTParser.jl:156 =# @default(ps, #= /home/mathow/.julia/packages/CSTParser/i8cPl/src/CSTParser.jl:156 =# @nocloser(ps, inwhere, parse_comma_sep(ps, args, false, true, true)))
        #= /home/mathow/.julia/packages/CSTParser/i8cPl/src/CSTParser.jl:158 =#
        if length(args) == 2 && (((ps.ws).kind != SemiColonWS || length(args) == 2 && (args[2]).typ === Block) && !((args[2]).typ === Parameters))
            #= /home/mathow/.julia/packages/CSTParser/i8cPl/src/CSTParser.jl:159 =#
            accept_rparen(ps, args)
         

...

 [8] signature(::Any, ::JuliaInterpreter.Frame, ::Any, ::Int64) at /home/mathow/.julia/packages/LoweredCodeUtils/6EZqd/src/LoweredCodeUtils.jl:146
 [9] #methoddef!#3(::Bool, ::Function, ::Any, ::Array{Any,1}, ::JuliaInterpreter.Frame, ::Any, ::Int64) at /home/mathow/.julia/packages/LoweredCodeUtils/6EZqd/src/LoweredCodeUtils.jl:420
 [10] (::getfield(LoweredCodeUtils, Symbol("#kw##methoddef!")))(::NamedTuple{(:define,),Tuple{Bool}}, ::typeof(LoweredCodeUtils.methoddef!), ::Function, ::Array{Any,1}, ::JuliaInterpreter.Frame, ::Expr, ::Int64) at ./none:0
 [11] #methods_by_execution!#14(::Bool, ::Bool, ::Function, ::Any, ::Revise.CodeTrackingMethodInfo, ::Dict{Module,Array{Expr,1}}, ::JuliaInterpreter.Frame) at /home/mathow/.julia/packages/Revise/agmgx/src/lowered.jl:81
 [12] #methods_by_execution! at ./none:0 [inlined]
 [13] #methods_by_execution!#9(::Base.Iterators.Pairs{Symbol,Bool,Tuple{Symbol},NamedTuple{(:define,),Tuple{Bool}}}, ::Function, ::Any, ::Revise.CodeTrackingMethodInfo, ::D

 [20] (::getfield(Revise, Symbol("##26#27")){String})() at /home/mathow/.julia/packages/Revise/agmgx/src/pkgs.jl:370
 [21] with_logstate(::getfield(Revise, Symbol("##26#27")){String}, ::Base.CoreLogging.LogState) at ./logging.jl:395
 [22] with_logger at ./logging.jl:491 [inlined]
 [23] watch_manifest(::String) at /home/mathow/.julia/packages/Revise/agmgx/src/pkgs.jl:349
 [24] (::Revise.Rescheduler{typeof(Revise.watch_manifest),Tuple{String}})() at /home/mathow/.julia/packages/Revise/agmgx/src/types.jl:235
while evaluating
methoddef!(#= /home/mathow/.julia/packages/LoweredCodeUtils/6EZqd/src/LoweredCodeUtils.jl:440 =# @nospecialize(recurse), signatures, frame::Frame, pc::Int; define=true) = begin
        #= /home/mathow/.julia/packages/LoweredCodeUtils/6EZqd/src/LoweredCodeUtils.jl:440 =#
        methoddef!(recurse, signatures, frame, pc_expr(frame, pc), pc; define=define)
    end
in module Main.LoweredCodeUtils
UndefVarError: Frame not defined
Stacktrace:
 [1] macro expansion at /home/

 [17] #instantiate_sigs!#60(::Bool, ::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}, ::Function, ::OrderedCollections.OrderedDict{Module,OrderedCollections.OrderedDict{Revise.RelocatableExpr,Union{Nothing, Array{Any,1}}}}) at /home/mathow/.julia/packages/Revise/agmgx/src/Revise.jl:352
 [18] instantiate_sigs! at /home/mathow/.julia/packages/Revise/agmgx/src/Revise.jl:349 [inlined]
 [19] maybe_parse_from_cache!(::Revise.PkgData, ::String) at /home/mathow/.julia/packages/Revise/agmgx/src/pkgs.jl:224
 [20] (::getfield(Revise, Symbol("##26#27")){String})() at /home/mathow/.julia/packages/Revise/agmgx/src/pkgs.jl:370
 [21] with_logstate(::getfield(Revise, Symbol("##26#27")){String}, ::Base.CoreLogging.LogState) at ./logging.jl:395
 [22] with_logger at ./logging.jl:491 [inlined]
 [23] watch_manifest(::String) at /home/mathow/.julia/packages/Revise/agmgx/src/pkgs.jl:349
 [24] (::Revise.Rescheduler{typeof(Revise.watch_manifest),Tuple{String}})() at /home/mathow/.julia/pac

 [21] with_logstate(::getfield(Revise, Symbol("##26#27")){String}, ::Base.CoreLogging.LogState) at ./logging.jl:395
 [22] with_logger at ./logging.jl:491 [inlined]
 [23] watch_manifest(::String) at /home/mathow/.julia/packages/Revise/agmgx/src/pkgs.jl:349
 [24] (::Revise.Rescheduler{typeof(Revise.watch_manifest),Tuple{String}})() at /home/mathow/.julia/packages/Revise/agmgx/src/types.jl:235
while evaluating
function methoddefs!(#= /home/mathow/.julia/packages/LoweredCodeUtils/6EZqd/src/LoweredCodeUtils.jl:459 =# @nospecialize(recurse), signatures, frame::Frame; define=true)
    #= /home/mathow/.julia/packages/LoweredCodeUtils/6EZqd/src/LoweredCodeUtils.jl:460 =#
    ret = methoddef!(recurse, signatures, frame; define=define)
    #= /home/mathow/.julia/packages/LoweredCodeUtils/6EZqd/src/LoweredCodeUtils.jl:461 =#
    pc = if ret === nothing
            ret
        else
            ret[1]
        end
    #= /home/mathow/.julia/packages/LoweredCodeUtils/6EZqd/src/LoweredCodeUtils.jl:462 

        #= /home/mathow/.julia/packages/LoweredCodeUtils/6EZqd/src/LoweredCodeUtils.jl:469 =#
        stmt = pc_expr(frame, pc)
        #= /home/mathow/.julia/packages/LoweredCodeUtils/6EZqd/src/LoweredCodeUtils.jl:470 =#
        if !(ismethod(stmt))
            #= /home/mathow/.julia/packages/LoweredCodeUtils/6EZqd/src/LoweredCodeUtils.jl:471 =#
            pc = next_until!(ismethod, recurse, frame, true)
        end
        #= /home/mathow/.julia/packages/LoweredCodeUtils/6EZqd/src/LoweredCodeUtils.jl:473 =#
        pc === nothing && break
        #= /home/mathow/.julia/packages/LoweredCodeUtils/6EZqd/src/LoweredCodeUtils.jl:474 =#
        ret = methoddef!(recurse, signatures, frame, pc; define=define)
        #= /home/mathow/.julia/packages/LoweredCodeUtils/6EZqd/src/LoweredCodeUtils.jl:475 =#
        pc = if ret === nothing
                ret
            else
                ret[1]
            end
    end
    #= /home/mathow/.julia/packages/LoweredCodeUtils/6EZqd/src/LoweredCodeUt

#### Compute measurement

In [22]:
mens = EnsembleState((N, NZ))
enkf.m(0.0, mens, ens)
Â = hcat(deepcopy(mens))

5×10 Array{Float64,2}:
 1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01
 1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01
 1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01
 1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01
 1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01

#### Compute deviation from measurement of the mean

In [23]:
Â′  = Â .- enkf.m(0.0, mean(ens))

5×10 Array{Float64,2}:
 -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04
 -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04
 -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04
 -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04
 -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04

#### Get actual measurements

In [24]:
zens = EnsembleState((N, NZ))
M = 1.01*ones(NZ)
enkf.z(0.0, zens)

EnsembleState{10,5,Float64}(Array{Float64,1}[[1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01]])

In [25]:
zens

EnsembleState{10,5,Float64}(Array{Float64,1}[[1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01]])

#### Perturb actual measurement

In [26]:
ϵ(zens)

EnsembleState{10,5,Float64}(Array{Float64,1}[[0.0729845, 0.360457, -0.595656, 0.716097, 0.919471], [2.74959, -0.0355739, 1.31031, 2.60403, 1.68868], [1.45465, 2.6402, 1.70232, 1.74599, -0.185757], [0.362484, 1.56684, 0.312665, 0.3011, 0.751375], [0.904745, 1.60629, 1.1223, -0.0596881, 1.59135], [0.728292, 1.84237, 1.72193, 1.3386, 1.17733], [1.2322, -0.00757869, 1.11979, 0.118813, 1.16315], [1.31645, 1.37846, 0.955742, 0.87712, 0.531026], [3.05504, 2.95354, 0.415102, 0.241278, 0.493672], [1.11261, 0.30206, -0.274355, 1.48479, 3.04099]])

#### Construct representer matrix and vectors

In [27]:
D = hcat(zens)

5×10 Array{Float64,2}:
  0.0729845   2.74959     1.45465   …  1.31645   3.05504    1.11261 
  0.360457   -0.0355739   2.6402       1.37846   2.95354    0.30206 
 -0.595656    1.31031     1.70232      0.955742  0.415102  -0.274355
  0.716097    2.60403     1.74599      0.87712   0.241278   1.48479 
  0.919471    1.68868    -0.185757     0.531026  0.493672   3.04099 

In [28]:
cov(ϵ)

5×5 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0

In [29]:
(Â′*Â′') + (N-1)*cov(ϵ)*I

5×5 Array{Float64,2}:
 9.016  0.016  0.016  0.016  0.016
 0.016  9.016  0.016  0.016  0.016
 0.016  0.016  9.016  0.016  0.016
 0.016  0.016  0.016  9.016  0.016
 0.016  0.016  0.016  0.016  9.016

In [30]:
@time b = ((Â′*Â′') + (N-1)*cov(ϵ)*I) \ (D - Â)

  2.355029 seconds (3.41 M allocations: 162.133 MiB, 4.94% gc time)


5×10 Array{Float64,2}:
 -0.103413     0.192648   0.0489536  …   0.0340488    0.226814    0.01128  
 -0.0714712   -0.116815   0.180682       0.0409387    0.215536   -0.0787806
 -0.177706     0.032728   0.0764729     -0.00603039  -0.0665127  -0.142827 
 -0.0319556    0.176475   0.0813245     -0.0147661   -0.0858264   0.0526339
 -0.00935846   0.074769  -0.133314      -0.0532211   -0.0577827   0.225545 

In [31]:
@time Bᵀb = (A′*Â′')*b

  0.010382 seconds (4.05 k allocations: 225.549 KiB)


3×10 Array{Float64,2}:
  1.76096e-18  -1.60853e-18  -1.13605e-18  …  -1.03819e-18  -3.03333e-19
 -8.9827e-19    8.20512e-19   5.79501e-19      5.29581e-19   1.5473e-19 
  1.36895e-18  -1.25044e-18  -8.83147e-19     -8.0707e-19   -2.35806e-19

#### Analysis step

In [32]:
ens  += cut(Bᵀb)

EnsembleState{10,3,Float64}(Array{Float64,1}[[-2.41591, 0.154592, 0.699435], [0.642284, 0.412356, 0.895485], [-0.627419, -0.999034, 1.13618], [1.06616, 0.200668, -1.81217], [2.52441, 0.76541, 0.950722], [-0.391827, 0.826463, -0.90361], [-1.25095, 0.344904, -2.04859], [1.11846, -0.31089, 0.157741], [-0.644366, 0.860481, -0.0346485], [-0.643458, -1.09681, 0.0802098]])

In [33]:
TP = TupleProduct((Normal(), TruncatedNormal(0.0,1.0,-0.1,0.1)))

TupleProduct{2,Continuous,Tuple{Normal{Float64},Truncated{Normal{Float64},Continuous}}}(
v: (Normal{Float64}(μ=0.0, σ=1.0), Truncated(Normal{Float64}(μ=0.0, σ=1.0), range=(-0.1, 0.1)))
)


In [34]:
typeof(TP)<:MultivariateDistribution

true